In [ ]:
import os
import warnings

import astropy.coordinates as coord
from astropy.io.fits.column import VerifyWarning
coord.galactocentric_frame_defaults.set('v4.0')
import astropy.table as at
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

# gala
import gala.coordinates as gc
import gala.dynamics as gd
import gala.integrate as gi
import gala.potential as gp
from gala.units import galactic

from pyia import GaiaData

# Shut up, FITS!
warnings.filterwarnings('ignore', category=VerifyWarning)

In [ ]:
g = GaiaData(at.Table.read('../data/all_stars_near_theOG.csv', format='ascii.ecsv'))
g = g[g.prob > 0.5]

tmass = at.Table.read('../data/TheOGGroup-2mass.csv')
g = GaiaData(at.join(g.data, tmass, keys='source_id', 
                     uniq_col_name='{col_name}{table_name}', table_names=['', '2']))

len(g)

In [ ]:
h = at.QTable.read('../data/hip_stars_near_theOG.csv', format='ascii.ecsv')
h = h[h['prob'] > 0.5]

In [ ]:
hbv = h['B-V'].value
hmhp = h['Hpmag'].value - coord.Distance(parallax=h['Plx']).distmod.value

h[(hmhp < 2.05) & (hbv < 0.5)]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.scatter(h['B-V'].value, 
           h['Hpmag'].value - coord.Distance(parallax=h['Plx']).distmod.value)

# boi = h['HIP'] == 71618
# ax.scatter(h['B-V'].value[boi], 
#            (h['Hpmag'].value - coord.Distance(parallax=h['Plx']).distmod.value)[boi],
#            s=300)

for age in np.arange(7.4, 8.2+1e-3, 0.2):
    iso = at.Table.read(f'/Users/apricewhelan/data/Isochrones/MIST/FeH_0.1_iso.fits')
    i = iso[np.isclose(iso['log10_isochrone_age_yr'], age)] 
    i = i[i['phase'] < 4]  # rgb or before
    ax.plot(i['Tycho_B']-i['Tycho_V'], i['Hipparcos_Hp'], 
            marker='', label=f'log10age=${age:.1f}$', zorder=-10)

ax.set_xlim(-1, 2)
ax.set_ylim(7.5, -2)

ax.set_xlabel('B-V')
ax.set_ylabel('M_Hp [mag]')

ax2 = ax.twinx()
ylim = ax.get_ylim()
fid_dm = coord.Distance(40*u.pc).distmod.value
ax2.set_ylim(ylim[0] + fid_dm, 
             ylim[1] + fid_dm)

fig.set_facecolor('w')

Now we have a bunch of candidate members!

In [ ]:
highprob_c = g.get_skycoord()

In [ ]:
# Absolute magnitude, color - not extinction corrected
mg = g.phot_g_mean_mag - g.distmod
bprp = g.phot_bp_mean_mag - g.phot_rp_mean_mag

# Absolute magnitude, color - extinction corrected
# mg = g.get_G0() - g.distmod
# bprp = g.get_BP0() - g.get_RP0()

mh = g.h_m
jmk = g.j_m - g.ks_m

In [ ]:
bperr = 1.086 * g.phot_bp_mean_flux_error / g.phot_bp_mean_flux
rperr = 1.086 * g.phot_rp_mean_flux_error / g.phot_rp_mean_flux
gerr = 1.086 * g.phot_g_mean_flux_error / g.phot_g_mean_flux

distmod_samples = g.get_error_samples(size=16_384)
distmod_err = np.std(distmod_samples.distmod, axis=1)
mgerr = np.sqrt(gerr**2 + distmod_err.value**2)
bprperr = np.sqrt(bperr**2 + rperr**2)

In [ ]:
plt.figure(figsize=(7, 4))
plt.scatter(highprob_c.barycentricmeanecliptic.lon.degree,
            highprob_c.barycentricmeanecliptic.lat.degree)
plt.xlim(360, 0)
plt.ylim(-90, 90)
plt.xlabel('ecliptic lon')
plt.ylabel('ecliptic lat')
plt.tight_layout()

plt.figure(figsize=(7, 4))
plt.scatter(highprob_c.galactic.l.degree,
            highprob_c.galactic.b.degree)
plt.xlim(360, 0)
plt.ylim(-90, 90)
plt.xlabel('Galactic lon')
plt.ylabel('Galactic lat')
plt.tight_layout()

plt.figure(figsize=(7, 4))
plt.scatter(highprob_c.ra.degree,
            highprob_c.dec.degree)
plt.xlim(360, 0)
plt.ylim(-90, 90)
plt.xlabel('RA')
plt.ylabel('Dec')
plt.tight_layout()

Color-magnitude diagrams with isochrones:

In [ ]:
def make_cmd(color_by=None, color_by_norm=None, color_by_label=None):
    
    if color_by is not None:
        fig, ax = plt.subplots(1, 1, figsize=(9.5, 8), 
                               constrained_layout=True)
        
        cs = ax.scatter(bprp, mg, c=color_by, 
                        cmap='cividis', 
                        norm=color_by_norm,
                        edgecolor='#666666', linewidth=0.5)
    else:
        fig, ax = plt.subplots(1, 1, figsize=(8, 8), 
                           constrained_layout=True)
        
        cs = ax.scatter(bprp, mg, c='k')

    ax.annotate('TIC 2749',
                (bprp[g.source_id == 1490845584382687232].value, 
                 mg[g.source_id == 1490845584382687232].value),
                xytext=(bprp[g.source_id == 1490845584382687232].value + 0.2, 
                        mg[g.source_id == 1490845584382687232].value - 0.4),
                arrowprops=dict(arrowstyle='->', color='#777777'), fontsize=14)

    ax.annotate('TOI 1807',
                (bprp[g.source_id == 1476485996883837184].value, 
                 mg[g.source_id == 1476485996883837184].value),
                xytext=(bprp[g.source_id == 1476485996883837184].value + 0.2, 
                        mg[g.source_id == 1476485996883837184].value - 0.4),
                arrowprops=dict(arrowstyle='->', color='#777777'), fontsize=14)

    ax.set_xlim(-0.5, 3)
    ax.set_ylim(12, -4)

    ax.set_xlabel(r'$G_{\rm BP}-G_{\rm RP}$')
    ax.set_ylabel('$M_G$')

    ax2 = ax.twinx()
    ylim = ax.get_ylim()
    fid_dm = coord.Distance(40*u.pc).distmod.value
    ax2.set_ylim(ylim[0] + fid_dm, 
                 ylim[1] + fid_dm)
    ax2.set_ylabel('~$G$ [mag]')
    
    if color_by is not None:
        cb = fig.colorbar(cs, ax=ax, aspect=40)
        if color_by_label is not None:
            cb.set_label(color_by_label)
    
    fig.set_facecolor('w')
    # fig.tight_layout()
    
    return fig, ax

In [ ]:
_ = make_cmd(color_by=g.radial_velocity_error.value,
             color_by_norm=mpl.colors.LogNorm(vmin=1e-1, vmax=1e1),
             color_by_label=f'RV err [{u.km/u.s:latex_inline}]')

In [ ]:
fig, ax = make_cmd()

for feh in np.arange(-0.2, 0.4+1e-3, 0.2):
    iso = at.Table.read(f'/Users/apricewhelan/data/Isochrones/MIST/FeH_{feh:.1f}_iso.fits')
    i = iso[np.isclose(iso['log10_isochrone_age_yr'], 7.8)]  # ~60 Myr
    i = i[i['phase'] < 4]  # rgb or before
    ax.plot(i['G_BP']-i['G_RP'], i['G'], marker='', label=f'[Fe/H]=${feh:.1f}$')

ax.legend(loc='upper right', fontsize=14)

In [ ]:
fig, ax = make_cmd()

for age in np.arange(7.6, 8.4+1e-3, 0.2):
    iso = at.Table.read(f'/Users/apricewhelan/data/Isochrones/MIST/FeH_0.1_iso.fits')
    i = iso[np.isclose(iso['log10_isochrone_age_yr'], age)] 
    i = i[i['phase'] < 4]  # rgb or before
    age = 10**age / 1e6
    ax.plot(i['G_BP']-i['G_RP'], i['G'], marker='', label=f'age $={age:.0f}' + r'~{\rm Myr}$')

ax.legend(loc='upper right')

How many brighter stars do we expect?

In [ ]:
from scipy.interpolate import interp1d
from tqdm.notebook import trange
import imf

In [ ]:
iso = at.Table.read(f'/Users/apricewhelan/data/Isochrones/MIST/FeH_-0.1_iso.fits')
i = iso[np.isclose(iso['log10_isochrone_age_yr'], 8.2)] 

fig, ax = make_cmd()
ax.plot(i['G_BP']-i['G_RP'], i['G'])
ax.set_xlim(-1, 3.5)
ax.set_ylim(20, -6)

In [ ]:
iso = at.Table.read(f'/Users/apricewhelan/data/Isochrones/MIST/FeH_-0.1_iso.fits')
i = iso[np.isclose(iso['log10_isochrone_age_yr'], 8.2)] 
i = i[i['phase'] <= 0]  # rgb or before

cluster = imf.make_cluster(10000, massfunc='salpeter')

sim_mags = np.zeros((len(cluster), 3))
for j, f in enumerate(['G', 'G_BP', 'G_RP']):
    interp = interp1d(i['initial_mass'], i[f], kind='cubic', 
                      bounds_error=False)
    sim_mags[:, j] = interp(cluster)
    
sim_mags = sim_mags[np.isfinite(sim_mags[:, 0])]
sim_mags.shape

In [ ]:
glim = (4, 7)
nsim = ((sim_mags[:, 0] > glim[0]) & (sim_mags[:, 0] < glim[1])).sum()
nmg = ((mg.value > glim[0]) & (mg.value < glim[1])).sum()
downsample_fac = int(round(nsim / nmg))
downsample_fac

In [ ]:
#fig, ax = plt.subplots(figsize=(6, 6))

fig, ax = make_cmd()

derp = sim_mags[::downsample_fac]
ax.scatter(derp[:,1]-derp[:,2], derp[:,0], color='tab:blue')

ax.axhspan(glim[0], glim[1], zorder=-100, 
           color='tab:green', alpha=0.2, lw=0)

# ax.set_xlim(-0.5, 3.)
# ax.set_ylim(12, -4)

In [ ]:
counts = {}

for massfunc in ['salpeter', 'kroupa']:
    counts[massfunc] = []
    for trial in trange(1024):
        cluster = imf.make_cluster(10000, massfunc=massfunc)

        sim_mags = np.zeros((len(cluster), 3))
        for j, f in enumerate(['G', 'G_BP', 'G_RP']):
            interp = interp1d(i['initial_mass'], i[f], kind='cubic', 
                              bounds_error=False)
            sim_mags[:, j] = interp(cluster)

        sim_mags = sim_mags[np.isfinite(sim_mags[:, 0])]

        nsim = ((sim_mags[:, 0] > glim[0]) & (sim_mags[:, 0] < glim[1])).sum()
        nmg = ((mg.value > glim[0]) & (mg.value < glim[1])).sum() - 6
        downsample_fac = int(round(nsim / nmg))

        sim_mags = sim_mags[::downsample_fac]
        lim = 6 - coord.Distance(40*u.pc).distmod.value
        counts[massfunc].append((sim_mags[:, 0] < lim).sum())

In [ ]:
fig = plt.figure(figsize=(6, 6))
for k in counts.keys():
    plt.hist(counts[k], bins=np.linspace(0, 50, 32), label=k, alpha=0.5)
plt.legend(loc='upper right', fontsize=14)
plt.xlabel('N stars G < 6 mag')
fig.set_facecolor('w')

---

In [ ]:
g.data[derp_mask].write('../data/TheOGGroup.csv', format='ascii.ecsv', overwrite=True)

In [ ]:
galcen = highprob_c.transform_to(coord.Galactocentric)

fig, axes = plt.subplots(1, 2, figsize=(10, 5),
                         sharex=True, sharey=True)

derp_mask = (galcen.z.to_value(u.pc) > 30)

ax = axes[0]
ax.scatter((galcen.x - -galcen.galcen_distance).to_value(u.pc)[derp_mask],
           galcen.y.to_value(u.pc)[derp_mask])
ax.set_xlim(-110, 110)
ax.set_ylim(-110, 110)
ax.set_xlabel('$x$ [pc]')
ax.set_ylabel('$y$ [pc]')

ax = axes[1]
ax.scatter((galcen.x - -galcen.galcen_distance).to_value(u.pc)[derp_mask],
           galcen.z.to_value(u.pc)[derp_mask] - galcen.z_sun.to_value(u.pc))

ax.set_xlabel('$x$ [pc]')
ax.set_ylabel('$z$ [pc]')

fig.tight_layout()
fig.set_facecolor('w')

In [ ]:
vxyz0 = galcen.galcen_v_sun.d_xyz

fig, axes = plt.subplots(1, 2, figsize=(10, 5),
                         sharex=True, sharey=True)

ax = axes[0]
ax.scatter((galcen.v_x - vxyz0[0]).to_value(u.km/u.s),
           (galcen.v_y - vxyz0[1]).to_value(u.km/u.s),
           alpha=0.5, linewidth=0)
ax.set_xlim(-30, 30)
ax.set_ylim(-30, 30)
ax.set_xlabel('$vx$')
ax.set_ylabel('$vy$')

ax = axes[1]
ax.scatter((galcen.v_x - vxyz0[0]).to_value(u.km/u.s),
           (galcen.v_z - vxyz0[2]).to_value(u.km/u.s),
           alpha=0.5, linewidth=0)

ax.set_xlabel('$vx$')
ax.set_ylabel('$vz$')

fig.tight_layout()
fig.set_facecolor('w')

In [ ]:
for x in g.source_id[np.argsort(g.phot_g_mean_mag)][:10]:
    print(f'Gaia DR2 {x}')

xmatch with APOGEE, LAMOST, etc.

In [ ]:
allstar = at.Table.read('/Users/apricewhelan/data/APOGEE_beta/allStar-r13-l33-58932beta.fits')

In [ ]:
np.isin(g.source_id, allstar['GAIA_SOURCE_ID']).sum()

In [ ]:
_ = make_cmd(color_by=np.isin(g.source_id, allstar['GAIA_SOURCE_ID']),
             color_by_norm=mpl.colors.Normalize(vmin=0, vmax=1),
             color_by_label=f'in APOGEE')

In [ ]:
stars = allstar[np.isin(allstar['GAIA_SOURCE_ID'], g.source_id)]
stars = stars[np.unique(stars['APOGEE_ID'], return_index=True)[1]]

In [ ]:
stars['VSINI']

In [ ]:
stars_100pc = allstar[allstar['GAIA_PARALLAX'] > coord.Distance(100*u.pc).parallax.value]
len(stars_100pc)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))

ax.errorbar(stars['M_H'], stars['ALPHA_M'],
            xerr=stars['M_H_ERR'],
            yerr=stars['ALPHA_M_ERR'],
            ls='none', marker='.')

ax.hist2d(stars_100pc['M_H'], stars_100pc['ALPHA_M'],
          bins=(np.arange(-0.5, 0.5+1e-4, 0.01),
                np.arange(-0.2, 0.2+1e-3, 0.005)),
          cmap='Blues', zorder=-10)

ax.set_xlim(-0.5, 0.5)
ax.set_ylim(-0.2, 0.2)

ax.set_xlabel('[M/H]')
ax.set_ylabel('[alpha/M]')

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))

ax.errorbar(stars['MG_FE'], 
            stars['MN_FE'],
            xerr=stars['MG_FE_ERR'],
            yerr=stars['MN_FE_ERR'],
            ls='none', marker='.')

ax.hist2d(stars_100pc['MG_FE'], 
          stars_100pc['MN_FE'],
          bins=(np.arange(-0.5, 0.5+1e-4, 0.01),
                np.arange(-0.2, 0.2+1e-3, 0.005)),
          cmap='Blues', zorder=-10)

ax.set_xlim(-0.5, 0.5)
ax.set_ylim(-0.2, 0.2)

# ax.set_xlabel('[M/H]')
# ax.set_ylabel('[alpha/M]')

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))

ax.errorbar(stars['MG_FE'], 
            stars['AL_FE'],
            xerr=stars['MG_FE_ERR'],
            yerr=stars['AL_FE_ERR'],
            ls='none', marker='.')

ax.hist2d(stars_100pc['MG_FE'], 
          stars_100pc['AL_FE'],
          bins=(np.arange(-0.5, 0.5+1e-4, 0.01),
                np.arange(-0.4, 0.2+1e-3, 0.005)),
          cmap='Blues', zorder=-10)

ax.set_xlim(-0.5, 0.5)
ax.set_ylim(-0.4, 0.2)

ax.set_xlabel('[Mg/Fe]')
ax.set_ylabel('[Al/Fe]')

TODO: xmatch with 2MASS and remake CMD